# 본문 스크래핑

In [ ]:
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm
import random
import urllib.request
from selenium.common.exceptions import NoSuchElementException, UnexpectedAlertPresentException, TimeoutException
from selenium import webdriver
from selenium.webdriver.common.by import By
# import chromedriver_auto


# 웹드라이버 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
# options.add_argument('headless') # 창 숨기기


: 

In [2]:
from user_agent import generate_user_agent, generate_navigator

print(generate_user_agent(device_type='desktop'))
print(generate_user_agent(os='win', device_type='desktop'))

navigator = generate_navigator()

random_user = generate_user_agent(os='win', device_type='desktop')
print(random_user)

Mozilla/5.0 (X11; Linux i686 on x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.81 Safari/537.36
Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.185 Safari/537.36
Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 5.1; WOW64; Trident/5.0)


In [3]:
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument(random_user)
driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = chrome_options)

driver.maximize_window()
driver.get("https://www.naver.com/")
driver.implicitly_wait(10)


C:\Users\user\AppData\Local\Temp\ipykernel_15388\1335855278.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path ='./chromedriver_win32/chromedriver.exe',options = chrome_options)


In [4]:
url_df = pd.read_csv("./나무문화/2020/나무문화_링크_2020_7월.csv", encoding='UTF-8')
print(url_df.shape, url_df.columns)
url_df


(14192, 3) Index(['url', 'title', 'date'], dtype='object')


,url,title,date
0,https://blog.naver.com/kittiepie/222047759528,7月 31日: 칠월의 마지막 밤은 매미 울음 속에,2020.07.31.
1,https://blog.naver.com/minghae00/222047758389,여름휴가 1일차:고모리저수지/경기도잣나무푸른숲/축령산...,2020.07.31.
2,https://blog.naver.com/chicbook/222047758534,한우리독서토론논술 초등부 8월 프로그램,2020.07.31.
3,https://blog.naver.com/wsc1025/222047756291,"아이와 구한말의 세 주인공들을 만나다! 고종, 흥선대원군...",2020.07.31.
4,https://blog.naver.com/misspark365/222047750066,강진 여행 1박2일 일정 + 나홀로 야간여행,2020.07.31.
...,...,...,...
14187,https://blog.naver.com/namsan_couple/222016467737,강릉 여행 경포대 가볼 곳 허균생가 가족과 함께 가기 좋은 곳,2020.07.01.
14188,https://blog.naver.com/correctgame/222017310122,1292. 2017.7.16. 대한민국 경상북도 경주시...,2020.07.01.
14189,https://blog.naver.com/jubalry/222016833799,넷마블 - 중국 관련주(게임주) & BTS(방탄소년단) 관련주,2020.07.01.
14190,https://blog.naver.com/dalloway7374/222017177943,김용남展 - 삶의 미학 - 심화,2020.07.01.


In [5]:
blog_links = url_df['url'].to_list()
print(len(blog_links))


14192


In [6]:
df = pd.DataFrame(columns = {'url','본문'})
df.to_csv("./나무문화/2020/나무문화_본문_2020_7월.csv", index=False, encoding='UTF-8')
df.to_excel("./나무문화/2020/나무문화_본문_2020_7월.xlsx", index=False, encoding='UTF-8')

In [7]:
len(df)

0

In [ ]:
blog_links[len(df):]

In [11]:
#### 삭제/비공개 링크 인덱스도 매번 바꿔줄 것!!! ###
# contents = []
# old_blogs = []
hide_link = []

for i in tqdm(blog_links[len(df):]):
    content_dic = {'url' : [], '본문' : []}
    content_dic['url'].append(i)
    driver.get(i)
    driver.implicitly_wait(10)
    try:
        driver.switch_to.frame("mainFrame")
    except TimeoutException as e:
        print('시간 초과 오류 : ', e)
        content_dic['본문'].append('수집불가능')
    except UnexpectedAlertPresentException as e:
    ### 이 오류 해결함. 에러 코드 파일에서 직접 임포트하면 된다!
        print(e)
                                                                            # 여기도 바꿔야 함
        print('{}번째 게시글은 비공개로 전환되었거나 게시판이 바뀜.'.format(blog_links.index(i)))
        hide_link.append(i)
        time.sleep(1.5)
        driver.switch_to.frame('mainFrame')
    time.sleep(1)
    try:
        a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
        content_dic['본문'].append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
        content_dic['본문'].append(a)
        #old_blogs.append(i)
    additional_df = pd.DataFrame(content_dic)
    df = df.append(additional_df, ignore_index = True)
    df.to_csv("./나무문화/2020/나무문화_본문_2020_7월.csv", index=False, encoding='UTF-8')
    df.to_excel("./나무문화/2020/나무문화_본문_2020_7월.xlsx", index=False, encoding='UTF-8')
    print(blog_links.index(i), df['본문'].tolist().index(a))
    # if blog_links.index(i) != df['본문'].tolist().index(a):
    #     break


  0%|          | 0/12699 [00:00<?, ?it/s]

C:\Users\user\AppData\Local\Temp\ipykernel_15388\4136842187.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(additional_df, ignore_index = True)
c:\Users\user\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'http://m.yes24.com/Goods/Detail/91072587
미션!%20황금%20카드를%20모아라!
남한과%20북한이%20오래전부터%20같이%20누려%20온%20문화와%20역사를%20들여다보며북한에%20대한%20공감대를%20넓히고%20한민족의%20가치를%20깨닫는%20이야기!『미션!%20황금%20카드를%20모아라!』는%20컴퓨터%20게임%20속에서%20만난%20남한%20소년과%20북한%20소녀가%20남북한의%20공통%20문화에%20관한%20특별한%20미션들을%20함께%20풀어%20나가는%20이야기를%20통해,%20남한과%20북한이%20오래전부터%20한반도에서%20한%20나라를%20이루며...
m.yes24.com
최근%20남북한에%20대한%20탈북자%20뉴스가%20연일%20나오면서%20아이와%20함께%20남한과%20북한에%20대해%20알아보기%20위해%20책을%20펼쳐%20보았어요.
아이들이%20좋아하는%20황금카드를%20모으기%20위해%20아슬아슬하게%20게임을%20즐기는%20모습이%20아이가%20더욱%20흥미있고%20긴장감%20있게%20책을%20함께%20읽을수%20있었던것%20같아요.
총%205가지의%20게임으로%20시작합니다.
미션!%20황금%20카드를%20모아라!
요약하기
***차례***
▶%20레벨1%20남북한%20유물%

1493 1493
1494 1491
1495 1495
1496 1491
1497 1497
1498 1498
1499 1499
1500 1500
1501 1501
1502 1502
1503 1503
1504 1504
1505 1505
1506 1506
1507 1507
1508 1508
1509 1509
1510 1510
1511 1511
1512 1512
1513 1513
1514 1514
1515 1515
1516 1516
1517 1517
1518 1518
1519 1519
1520 1520
1521 1521
1522 1522
1523 1523
1524 1524
1525 1525
1526 1526
1527 1527
1528 1528
1529 1529
1530 1530
1531 1531
1532 1532
1533 1533
1534 1534
1535 1535
1536 1536
1537 1537
1538 1538
1539 1539
1540 1540
1541 1541
1542 1542
1543 1543
1544 1544
1545 1545
1546 1546
1547 1547
1548 1548
1549 1549
1550 1550
1551 1551
1552 1552
1553 1553
1554 1554
1555 1555
1556 1556
1557 1557
1558 1558
1559 1559
1560 1560
1561 1561
1562 1562
1563 1563
1564 1564
1565 1565
1566 1566


c:\Users\user\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://youtu.be/sXKhkc4qtx4?t=107
제주도점집%20엄지보살%20소띠%20호랑이띠%20띠궁합%20띠별궁합
안녕하세요%20제주도점집%20엄지보살%20인사드립니다!
의리가%20강하고%20속이%20따뜻하고%20정이%20많으며
빈틈이%20없고%20성실한%20소띠와
정직함과%20열정이%20가득한%20호랑이띠!
과연%20이%20둘의%20만남은%20띠%20궁합에서%20어떠한%20결과를%20보여줄지
오늘은%20소띠와%20호랑이띠의%20성격과%20띠궁합을%20준비해보았습니다
소띠%20성격
인내심이%20강하고%20믿음과%20의리가%20강하고%20두터운%20소띠는
거짓말을%20거의%20하지%20않고%20겉은%20엄격해%20보이고%20세%20보여도
속은%20따뜻하고%20정이%20많은%20띠라고%20할%20수%20있겠습니다
소띠는%20정직하고%20근실한%20모습도%20가지고%20있으며
입이%20무겁고%20끈질기게%20노력하고%20성실하게%20나아가는%20행동파입니다
다른%20사람에게%20지기%20싫어하는%20기질과%20뚝심으로%20목표에%20도달하는%20모습을%20보입니다
또한%20인간적인%20매력으로%20주변%20사람들을%20매료시키지만
때로는%20사랑에%20약하고%20겁이%20많아%20보수적인%20성향이%20강합니다
소띠는%20쉴틈없이%20부지런하고%20올바른%20것을%20보고
바르게%20행동하며%20천성적으로%20마음이%20착하기%20때문에
주변%20사람들에게%20피해를%20끼치지%20않고
자신의%20일에만%20집중할%20것입니다
특히%20자신의%20인생에%20대한%20치밀한%20계획을%20세워서
한보씩%20정진하니%20대기만성형입니다
아무것도%20없이%20시작한%20삶이라도
중년기에는%20넉넉한%20재물을%20갖추게%20될%20것입니다
호랑이띠%20성격
힘이%20넘치고%20누군

1567 1567
1568 1568
1569 1569
1570 1570
1571 1571
1572 1572
1573 1573
1574 1574
1575 1575
1576 1576
1577 1577
1578 1578
1579 1579
1580 1580
1581 1581
1582 1582
1583 1583
1584 1584
1585 1585
1586 1586
1587 1587
1588 1588
1589 1589
1590 1590
1591 1591
1592 1592
1593 1593
1594 1594
1595 1595
1596 1596
1597 1597
1598 1598
1599 1599
1600 1600
1601 1601
1602 1602
1603 1603
1604 1604
1605 1605
1606 1606
1607 1607
1608 1608
1609 1609
1610 1610
1611 1611
1612 1612
1613 1613
1614 1614
1615 1615
1616 1616
1617 1617
1618 1618
1619 1619
1620 1620
1621 1621
1622 1622
1623 1623
1624 1624
1625 1625
1626 1626
1627 1627
1628 1628
1629 1629
1630 1630
1631 1631
1632 1632
1633 1633
1634 1634
1635 1635
1636 1636
1637 1637
1638 1638
1639 1639
1640 1640
1641 1641
1642 1642
1643 1643
1644 1644
1645 1645
1646 1646
1647 1647
1648 1648
1649 1649
1650 1650
1651 1651
1652 1652
1653 1653
1654 1654
1655 1655
1656 1656
1657 1657
1658 1658
1659 1659
1660 1660
1661 1661
1662 1662
1663 1663


...

1746 1746
1747 1747
1748 1748
1749 1749


c:\Users\user\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://healing.seogwipo.go.kr/healing/healingintro.htm
서귀포시%20산림휴양림
개요%20위%20%20%20%20%20치%20:%20제주특별자치도%20서귀포시%20산록남로%202271번지%20조성면적%20:%20174ha%20관리운영%20:%20서귀포시%20산림휴양관리소%20사전예약%201일%20수용인원%20:%20주중%20300명%20주말%20600명으로%20제한(인원초과%20시%20입장불가)%20주차대수%20:%2078대%20(대형버스2대)%20개장일%20:%202016.%206.%2026%20이용문의%20:%20산림휴양관리소%20064)760%20-%203067%20~%208%20치유의%20숲이란?%20인체의%20면역력을%20높이고%20건강%20증진을%20목적으로%20산림의%20다양한%20환경요소를%20활용할%20수%20있도록%20조성한%20산림입니다.%20치유의%20숲은%20다양한%20산림의%20환경요소를%20활용하여%20산림치유지도사가%20산림치유...
healing.seogwipo.go.kr
예약제로%20운영되고%20있어요.%20자세한%20내용은%20홈페이지를%20참고하시면%20되겠습니다.
숲길만%20걸을%20수도%20있지만%20산림치유프로그램을%20이용할%20수도%20있답니다.
but,%20현재는%20코로나로%20인해%20잠정%20중단%20상태입니다.
차롱치유밥상도%20중단되었어요.
차롱치유밥상이란?%20차롱은%20제주에서%20음식을%20담기%20위해%20대나무로%20만들어%20사용하던%20바구니예요.
호근마을%20주민들이%20제주%20향토음식을%20만들어%20담아%20제공해주시는%20건강한%20밥상입니다.
코로나%20사태가%20끝나고%20나면%20산림치유프로그램%20체험하시고%20차롱밥상%20드시는걸%20추천드려요!
건강하고%20

1750 1750
1751 1751
1752 1752
1753 1753
1754 1754
1755 1755
1756 1756
1757 1757
1758 1758
1759 1759
1760 1760
1761 1761
1762 1762
1763 1763
1764 1764
1765 1765
1766 1766
1767 1767
1768 1768
1769 1769
1770 1770
1771 1771
1772 1772
1773 1773
1774 1774
1775 1775
1776 1776
1777 1777
1778 1778
1779 1779
1780 1780
1781 1781
1782 1782
1783 1783
1784 1784
1785 1785
1786 1786
1787 1787
1788 1788
1789 1789
1790 1790
1791 1791
1792 1792
1793 1793
1794 1794
1795 1795
1796 1796
1797 1797
1798 1798
1799 1799
1800 1800
1801 1801
1802 1802
1803 1803
1804 1804
1805 1805
1806 1806
1807 1807
1808 1808
1809 1809
1810 1810
1811 1811
1812 1812
1813 1813
1814 1814
1815 1815
1816 1816
1817 1817
1818 1818
1819 1819
1820 1820
1821 1821
1822 1822
1823 1823
1824 1824
1825 1825
1826 1826
1827 1827
1828 1828
1829 1829
1830 1830
1831 1831
1832 1832
1833 1833
1834 1834
1835 1835
1836 1836
1837 1837
1838 1838
1839 1839
1840 1840
1841 1841
1842 1842
1843 1843
1844 1844
1845 1845
1846 1846
1847 1847
1848 1848
1849 1849


: 

: 

In [9]:
df

,본문,url
0,1. 이번 7월은 허연의 칠월을 읽지 않았고 루카 구아다니노의 아이엠러브도 보지 않...,https://blog.naver.com/kittiepie/222047759528
1,2020년 7월 31일\n여름휴가 1일차\n일정: 포천 고모리 욕쟁이할머니집 점심 ...,https://blog.naver.com/minghae00/222047758389
2,생각하는 나무 어휘리더 1 (초등1)\n1차시 자신만만 읽기 • 쓰기\n《말짱 도루...,https://blog.naver.com/chicbook/222047758534
3,"반갑습니다. 오늘은 인사동 주변 볼거리를 소개합니다. 바로 운현궁, 천도교중앙대교당...",https://blog.naver.com/wsc1025/222047756291
4,"강진 여행 1박2일 일정 + 나홀로아간여행\n글, 사진, 영상 ⓒ박매력\n대한민국 ...",https://blog.naver.com/misspark365/222047750066
...,...,...
1488,미스트랄은 프랑스 남부 산악지방에서 리옹 만 쪽으로 부는 북서풍을 말한다. 미스트랄...,https://blog.naver.com/neuro412/222044475909
1489,나의 문화유산답사기를 지은\n유홍준 교수는 산사순례 편에서\n남한 땅의 대 명찰 중...,https://blog.naver.com/k9897/222043422208
1490,"안녕하세요, 경매아라누리 운영하는\n리치인베스트 선임이사 최호규입니다.\n오늘 소개...",https://blog.naver.com/chlghrb1741/222044471316
1491,,https://blog.naver.com/2479v/222044470606


In [9]:
print(len(contents))
#print(len(old_blogs))
print(len(hide_link))


214
28
0
